# FTS (Focused Test Statistics) Implementation with xRooFit
## A Modern Approach to Statistical Hypothesis Testing in Particle Physics

This notebook implements the **Focused Test Statistics (FTS)** method, a novel approach that enhances statistical sensitivity in specific parameter regions of interest. Unlike traditional likelihood ratio tests that treat all parameter values equally, FTS uses a "focus function" to concentrate statistical power where physics considerations suggest signals are most likely to appear.

### Key References:
- **FTS Paper**: [On Focusing Statistical Power for Searches and Measurements in Particle Physics](https://arxiv.org/pdf/2507.17831)
- **Demo Code**: [FocusedTestStatDemo.html](https://will.web.cern.ch/FocusedTestStatDemo.html)

### What You'll Learn:
1. How to set up a statistical model using xRooFit (CERN's ROOT extension)
2. How to implement the FTS algorithm with numerical stability
3. How to generate toy datasets for p-value calculation
4. How to compare FTS with traditional likelihood ratio tests

### Mathematical Foundation:
The FTS test statistic is defined as:
$$T_f(D; \mu_0) = -2 \log\left(\frac{L(\mu_0|D)}{\int L(\mu|D) f(\mu) d\mu}\right)$$

Where:
- $L(\mu|D)$ is the likelihood function
- $f(\mu)$ is the focus function (typically a truncated Gaussian)
- $\mu_0$ is the null hypothesis value
- The integral represents a Bayesian-weighted average

In [ ]:
# ==============================================================================
# SECTION 1: Environment Setup and Statistical Model Creation
# ==============================================================================
# This section sets up the computational environment and creates a statistical
# model representing a typical particle physics counting experiment with:
# - Signal and background processes
# - Systematic uncertainties
# - Observed data (with some signal injected for realistic testing)
# ==============================================================================

import sys
import os
import time
import math
from typing import Dict, Tuple, Optional, Callable
from collections import defaultdict

print("=" * 70)
print("FTS Implementation - Environment Setup")
print("=" * 70)

# Track setup time for performance monitoring
setup_start = time.time()

# Configure threading for reproducible results
# Single-threaded execution ensures consistency across runs
os.environ['OMP_NUM_THREADS'] = os.environ.get('OMP_NUM_THREADS', '1')

# ------------------------------------------------------------------------------
# ROOT Configuration
# ------------------------------------------------------------------------------
# ROOT is CERN's data analysis framework, required for xRooFit
# We need to add ROOT's Python bindings to the system path

# Standard ROOT installation paths for macOS (via Homebrew)
brew_root_lib = "/opt/homebrew/opt/root/lib"
brew_root_py = "/opt/homebrew/opt/root/lib/root"

# Add ROOT paths to Python's module search path
for p in [brew_root_lib, brew_root_py]:
    if p and p not in sys.path:
        sys.path.insert(0, p)

# Set PYTHONPATH environment variable for ROOT
os.environ['PYTHONPATH'] = f"/opt/homebrew/Cellar/root/6.34.08_1/lib/root:{os.environ.get('PYTHONPATH', '')}"

# Import ROOT - this will fail if ROOT is not installed
import ROOT

# ------------------------------------------------------------------------------
# Project Path Configuration
# ------------------------------------------------------------------------------
# Add project directories to Python path for importing custom modules

# Get the parent directory (project root)
parent_dir = os.path.dirname(os.getcwd())
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)
print(f"✅ Added project root to path: {parent_dir}")

# Add examples directory for html_exact_compat module
examples_dir = os.path.join(parent_dir, 'examples')
if examples_dir not in sys.path:
    sys.path.insert(0, examples_dir)
print(f"✅ Added examples to path: {examples_dir}")

# Import xRooFit compatibility layer
# This module sets up the custom xRooFit build if available
import html_exact_compat  
print(f"✅ xRooFit compatibility layer loaded")

# Disable interactive JavaScript ROOT plots in Jupyter
%jsroot off

# ------------------------------------------------------------------------------
# ROOT Visual Style Configuration
# ------------------------------------------------------------------------------
# Configure ROOT's plotting style for publication-quality figures

ROOT.gStyle.SetTitleSize(0.035, "XYZ")      # Axis title size
ROOT.gStyle.SetLabelSize(0.03, "XYZ")       # Axis label size
ROOT.gStyle.SetTitleFontSize(0.04)          # Overall title font size
ROOT.gStyle.SetLegendTextSize(0.03)         # Legend text size
ROOT.gStyle.SetOptStat(False)               # Disable statistics box on plots

# ------------------------------------------------------------------------------
# Minimizer Configuration
# ------------------------------------------------------------------------------
# Configure MINUIT (ROOT's minimization engine) for optimal performance

# Strategy 1: Balance between speed and reliability
ROOT.Math.MinimizerOptions.SetDefaultStrategy(1)

# Tolerance: Convergence criterion for fits
ROOT.Math.MinimizerOptions.SetDefaultTolerance(1e-3)

# Initialize performance monitoring dictionary
perf_stats = defaultdict(list)

print("\n" + "=" * 70)
print("Creating Statistical Model with xRooFit")
print("=" * 70)

# ==============================================================================
# Statistical Model Creation
# ==============================================================================
# We create a simple counting experiment model with:
# - 3 bins (representing different analysis regions or energy ranges)
# - Background process with known expected rates
# - Signal process scaled by parameter of interest μ
# - 10% systematic uncertainty on background

# Create the main workspace using xRooFit
# The workspace contains all PDFs, parameters, and datasets
w = ROOT.xRooNode("RooWorkspace", "combined", "FTS demonstration workspace")

# Define the signal region (SR) with 3 bins
# This could represent different categories or phase space regions
w["pdfs/simPdf/SR"].SetXaxis(3, 0, 3)  # 3 bins: [0,1), [1,2), [2,3)

# ------------------------------------------------------------------------------
# Background Configuration
# ------------------------------------------------------------------------------
# Set expected background rates for each bin
# These would typically come from control regions or simulations
w["pdfs/simPdf/SR/bkg"].SetBinContent(1, 15)  # Bin 1: 15 expected events
w["pdfs/simPdf/SR/bkg"].SetBinContent(2, 20)  # Bin 2: 20 expected events
w["pdfs/simPdf/SR/bkg"].SetBinContent(3, 17)  # Bin 3: 17 expected events

# Add systematic uncertainty: 10% normalization uncertainty
# The parameter alpha_sys controls the systematic shift
# When alpha_sys = +1, background increases by 10%
w["pdfs/simPdf/SR/bkg"].SetBinContent(1, 15*1.1, "alpha_sys", 1)
w["pdfs/simPdf/SR/bkg"].SetBinContent(2, 20*1.1, "alpha_sys", 1)
w["pdfs/simPdf/SR/bkg"].SetBinContent(3, 17*1.1, "alpha_sys", 1)

# Constrain the systematic parameter with a Gaussian (mean=0, sigma=1)
# This implements the systematic uncertainty in the likelihood
w["pdfs/simPdf"].pars()["alpha_sys"].Constrain("normal")

# ------------------------------------------------------------------------------
# Signal Configuration
# ------------------------------------------------------------------------------
# Set expected signal yields for each bin (before scaling by μ)
# These represent the signal shape/distribution
w["pdfs/simPdf/SR/sig"].SetBinContent(1, 1)  # Relative signal in bin 1
w["pdfs/simPdf/SR/sig"].SetBinContent(2, 1)  # Relative signal in bin 2
w["pdfs/simPdf/SR/sig"].SetBinContent(3, 1)  # Relative signal in bin 3

# The signal is scaled by the parameter of interest μ
# μ=0: background-only hypothesis
# μ=1: signal at nominal strength
# μ>1: enhanced signal
w["pdfs/simPdf/SR/sig"].Multiply("mu")

# ------------------------------------------------------------------------------
# Parameter of Interest Configuration
# ------------------------------------------------------------------------------
# Mark μ as the parameter of interest (POI)
w.pars()["mu"].setAttribute("poi")

# Set allowed range for μ (important for integration bounds)
w.pars()["mu"].setRange(-100, 100)

# ------------------------------------------------------------------------------
# Generate Observed Data
# ------------------------------------------------------------------------------
# For demonstration, we inject some signal (μ=2) to create realistic data
# In real analysis, this would be actual detector data

print("\nGenerating observed dataset with injected signal...")
w.pars()["mu"].setVal(2.0)  # Set true signal strength
toy_data = w["pdfs/simPdf"].generate()  # Generate Poisson-distributed data
toy_data.get().SetNameTitle("obsData", "Observed Data")
w.Add(toy_data)  # Add to workspace
w.pars()["mu"].setVal(0.0)  # Reset μ for analysis

# ------------------------------------------------------------------------------
# Visualize the Model
# ------------------------------------------------------------------------------
# Create a plot showing the model and observed data
print("Creating visualization of model and data...")
w["pdfs/simPdf"].Draw("eratio")  # Draw PDF with error bands
w["datasets/obsData"].Draw("same")  # Overlay observed data
ROOT.gPad.GetCanvas().Draw()

# Report setup completion
setup_time = time.time() - setup_start
perf_stats['setup_time'].append(setup_time)

print("\n" + "=" * 70)
print(f"✅ Statistical model created successfully in {setup_time:.2f}s")
print(f"   • 3 bins with background rates: [15, 20, 17] events")
print(f"   • Signal injection: μ = 2.0")
print(f"   • Systematic uncertainty: 10% on background")
print("=" * 70)

In [ ]:
# ==============================================================================
# SECTION 2: FTS Core Implementation - Focus Functions and NLL Calculator
# ==============================================================================
# This section implements the core components of the Focused Test Statistics:
# 1. Focus function (weights different μ values)
# 2. NLL calculator with caching for performance
# 3. Numerical integration routines with stability enhancements
# ==============================================================================

import numpy as np

print("\n" + "=" * 70)
print("Loading FTS Core Components")
print("=" * 70)

# ==============================================================================
# Focus Function Class
# ==============================================================================
# The focus function f(μ) determines how much weight each parameter value
# receives in the FTS denominator integral. By choosing an appropriate focus
# function, we can enhance sensitivity in physically motivated regions.

class ProductionFocusFunction:
    """
    Focus function for FTS implementation.
    
    The focus function weights different values of the parameter of interest (μ)
    in the denominator of the FTS. Common choices include:
    - Gaussian: Focuses on a specific value with smooth falloff
    - Top-hat: Uniform weight within a range, zero outside
    
    Physics motivation: If theoretical predictions or previous experiments
    suggest a signal around μ=1 with uncertainty σ=0.5, we can use a Gaussian
    focus to concentrate statistical power in that region.
    """
    
    def __init__(
        self,
        mu_focus: float = 0.0,      # Center of the focus region
        sigma_focus: float = 1.5,    # Width of the focus region
        normalize: bool = True,      # Whether to normalize the focus function
        weight_type: str = 'gaussian',  # Type of focus function
        theta_lo: Optional[float] = None,  # Lower bound for truncation
        theta_hi: Optional[float] = None,  # Upper bound for truncation
    ):
        self.mu_focus = mu_focus
        self.sigma_focus = sigma_focus
        self.weight_type = weight_type
        self.normalize = normalize
        self.theta_lo = theta_lo
        self.theta_hi = theta_hi

    def weight(self, mu: float) -> float:
        """
        Compute the focus weight for a given μ value.
        
        The weight determines how much this μ value contributes to the
        denominator integral in the FTS formula.
        """
        if self.weight_type == 'gaussian':
            # Gaussian focus: exp(-0.5 * ((μ - μ_focus) / σ_focus)²)
            # Concentrates weight near μ_focus with width σ_focus
            z = (mu - self.mu_focus) / self.sigma_focus
            val = math.exp(-0.5 * z * z)
            
            # Apply truncation bounds if specified
            # This can be used to enforce physical constraints (e.g., μ ≥ 0)
            if self.theta_lo is not None and mu < self.theta_lo:
                return 0.0
            if self.theta_hi is not None and mu > self.theta_hi:
                return 0.0
            return val
            
        elif self.weight_type == 'tophat':
            # Top-hat (uniform) focus within range, zero outside
            # Useful for testing sensitivity in a specific interval
            return 1.0 if abs(mu - self.mu_focus) <= self.sigma_focus else 0.0
        else:
            raise ValueError(f"Unknown weight_type: {self.weight_type}")

    def get_dynamic_integration_range(self, mu0: float, n_sigma: float = 5.0) -> Tuple[float, float]:
        """
        Determine optimal integration range based on focus and null hypothesis.
        
        The integration range should:
        1. Cover the focus region (where f(μ) is significant)
        2. Include the null hypothesis value μ₀
        3. Be wide enough for accurate numerical integration
        
        Strategy: Center between μ₀ and μ_focus, extend by n_sigma * σ_focus
        """
        center = (self.mu_focus + mu0) / 2.0
        half_range = n_sigma * self.sigma_focus + abs(mu0 - self.mu_focus)
        lo = center - half_range
        hi = center + half_range
        
        # Apply bounds if specified
        if self.theta_lo is not None:
            lo = max(lo, self.theta_lo)
        if self.theta_hi is not None:
            hi = min(hi, self.theta_hi)
        return (lo, hi)

    def get_uniform_grid(self, mu0: float, n_points: int = 401, n_sigma: float = 5.0) -> np.ndarray:
        """
        Create a uniform grid for numerical integration.
        
        For accurate integration, we need:
        - Odd number of points (for Simpson's rule)
        - Sufficient density in the focus region
        - Coverage of the full integration range
        """
        # Ensure odd number of points for Simpson's rule
        if n_points % 2 == 0:
            n_points += 1
            
        # Get integration bounds
        lo, hi = self.get_dynamic_integration_range(mu0, n_sigma=n_sigma)
        
        # Protection against degenerate cases
        if not math.isfinite(lo) or not math.isfinite(hi) or hi <= lo:
            hi = lo + max(1e-6, abs(self.sigma_focus))
            
        return np.linspace(lo, hi, n_points)

# ==============================================================================
# Cached NLL Calculator
# ==============================================================================
# Computing the negative log-likelihood (NLL) for each μ value requires
# a fit to optimize nuisance parameters. This is computationally expensive,
# so we cache results to avoid redundant calculations.

class CachedNLLCalculator:
    """
    Efficient NLL calculator with caching.
    
    For FTS, we need to evaluate the likelihood at many μ values.
    Each evaluation requires:
    1. Fixing μ to the test value
    2. Minimizing over nuisance parameters (profiling)
    3. Extracting the minimum NLL value
    
    Caching Strategy:
    - Store (dataset, μ) -> NLL mappings
    - Reuse values when possible (especially for toys)
    - Track cache performance for optimization
    """
    
    def __init__(self, workspace):
        self.workspace = workspace
        self.cache: Dict[Tuple[str, float], float] = {}
        self.cache_hits = 0
        self.cache_misses = 0
        self.failed_fits = 0

    def get_nll_at_mu(self, dataset: str, mu_value: float, use_cache: bool = True) -> Optional[float]:
        """
        Compute profiled NLL for given μ and dataset.
        
        This implements the profile likelihood:
        NLL(μ) = min_θ [-log L(μ, θ | data)]
        where θ represents nuisance parameters.
        """
        # Create cache key (round μ to avoid floating point issues)
        cache_key = (dataset, round(float(mu_value), 6))
        
        # Check cache first
        if use_cache and cache_key in self.cache:
            self.cache_hits += 1
            return self.cache[cache_key]

        self.cache_misses += 1

        # Get parameter of interest
        poi = self.workspace.pars()["mu"]
        old_val = poi.getVal()
        old_const = poi.isConstant()
        
        try:
            # Fix μ to test value
            poi.setVal(float(mu_value))
            poi.setConstant(True)  # Fix during minimization

            # Create NLL object and minimize over nuisance parameters
            nll = self.workspace["pdfs/simPdf"].nll(dataset)
            fitres = nll.minimize()
            
            # Check fit quality
            if fitres.status() >= 4:  # Status 4+ indicates problems
                self.failed_fits += 1
                return None

            # Extract minimum NLL value
            nll_value = float(fitres.minNll())
            
            # Validate result
            if not math.isfinite(nll_value):
                self.failed_fits += 1
                return None

            # Cache successful result
            if use_cache:
                self.cache[cache_key] = nll_value
            return nll_value
            
        except Exception:
            self.failed_fits += 1
            return None
        finally:
            # Always restore original parameter state
            poi.setVal(old_val)
            poi.setConstant(old_const)

    def get_cache_stats(self) -> Dict:
        """Return cache performance statistics."""
        total = self.cache_hits + self.cache_misses
        hit_rate = self.cache_hits / total if total > 0 else 0.0
        return {
            'hits': self.cache_hits,
            'misses': self.cache_misses,
            'hit_rate': hit_rate,
            'cache_size': len(self.cache),
            'failed_fits': self.failed_fits,
        }

# ==============================================================================
# Numerical Integration Utilities
# ==============================================================================
# The FTS denominator requires integrating L(μ|D) * f(μ) over μ.
# We use numerical techniques with stability enhancements.

def _build_weight_grid(mu_grid: np.ndarray, focus: ProductionFocusFunction) -> np.ndarray:
    """
    Build normalized weight grid from focus function.
    
    The weights need to be:
    1. Evaluated at each grid point
    2. Normalized (if requested) so ∫f(μ)dμ = 1
    3. Non-negative (enforced for numerical stability)
    """
    # Evaluate focus function at each grid point
    w = np.array([focus.weight(mu) for mu in mu_grid], dtype=float)
    
    # Normalize if requested (makes FTS invariant to focus scaling)
    if focus.normalize:
        # Numerical integration using trapezoidal rule
        Z = np.trapz(w, mu_grid)
        if Z <= 0 or not math.isfinite(Z):
            Z = 1.0  # Fallback for degenerate cases
        w = w / Z
    
    # Ensure non-negative weights
    w = np.clip(w, 0.0, np.inf)
    return w

def _log_denom_from_profile(mu_grid: np.ndarray, nll_grid: np.ndarray, w_grid: np.ndarray) -> float:
    """
    Compute log of denominator integral using numerically stable methods.
    
    We need to compute: log(∫ L(μ|D) * f(μ) dμ)
    
    Challenge: L(μ|D) = exp(-NLL(μ)) can vary by many orders of magnitude
    Solution: Use log-sum-exp trick for numerical stability
    
    The log-sum-exp trick computes log(Σ exp(aᵢ)) as:
    log(Σ exp(aᵢ)) = aₘₐₓ + log(Σ exp(aᵢ - aₘₐₓ))
    This prevents overflow/underflow in exponentials.
    """
    # Convert to log space: log(L * f) = log(L) + log(f) = -NLL + log(f)
    a = -nll_grid + np.log(np.maximum(w_grid, 1e-300))
    
    # Apply log-sum-exp trick
    amax = float(np.max(a))
    y = np.exp(a - amax)  # Scaled values for integration

    # Choose integration method based on grid uniformity
    dh = np.diff(mu_grid)
    uniform = np.allclose(dh, dh[0], rtol=0.05, atol=0)

    if uniform and len(mu_grid) >= 3 and (len(mu_grid) % 2 == 1):
        # Simpson's rule (higher accuracy for smooth functions)
        # ∫f dx ≈ (h/3)[f₀ + 4f₁ + 2f₂ + 4f₃ + ... + fₙ]
        h = dh[0]
        S = y[0] + y[-1] + 4.0 * np.sum(y[1:-1:2]) + 2.0 * np.sum(y[2:-2:2])
        integral = S * h / 3.0
    else:
        # Trapezoidal rule (more robust for non-uniform grids)
        integral = np.trapz(y, mu_grid)

    # Ensure positive integral
    integral = max(float(integral), 1e-300)
    
    # Return log of integral
    return amax + math.log(integral)

def compute_production_fts_ts(
    nll_calc: CachedNLLCalculator,
    dataset: str,
    mu0: float,
    focus: ProductionFocusFunction,
    integration_points: int = 401,
) -> Optional[float]:
    """
    Compute the FTS test statistic for given data and null hypothesis.
    
    This is the main FTS calculation implementing:
    T_f(D; μ₀) = -2 * [log L(μ₀|D) - log ∫ L(μ|D) f(μ) dμ]
    
    Steps:
    1. Compute numerator: L(μ₀|D) via profile likelihood
    2. Build integration grid covering focus region
    3. Evaluate L(μ|D) at each grid point
    4. Compute weighted integral with focus function
    5. Return test statistic
    """
    ts_start = time.time()

    # Step 1: Compute numerator (null hypothesis likelihood)
    nll_mu0 = nll_calc.get_nll_at_mu(dataset, mu0)
    if nll_mu0 is None:
        return None
    lnL_numerator = -float(nll_mu0)  # Convert NLL to log-likelihood

    # Step 2: Create integration grid
    mu_grid = focus.get_uniform_grid(mu0, n_points=integration_points, n_sigma=5.0)

    # Step 3: Evaluate likelihood across grid
    nll_vals = []
    valid_mu = []
    for mu in mu_grid:
        nll_mu = nll_calc.get_nll_at_mu(dataset, float(mu))
        if nll_mu is None:
            continue  # Skip failed fits
        valid_mu.append(float(mu))
        nll_vals.append(float(nll_mu))

    # Require minimum number of successful evaluations
    if len(nll_vals) < 11:
        return None

    valid_mu = np.asarray(valid_mu, dtype=float)
    nll_vals = np.asarray(nll_vals, dtype=float)

    # Step 4: Build weight grid and compute integral
    w_grid = _build_weight_grid(valid_mu, focus)
    ln_denominator = _log_denom_from_profile(valid_mu, nll_vals, w_grid)

    # Step 5: Compute FTS test statistic
    fts_ts = -2.0 * (lnL_numerator - ln_denominator)

    # Track performance
    perf_stats['ts_calculation_time'].append(time.time() - ts_start)
    return float(fts_ts)

# Report successful loading
print("✅ Focus function class loaded")
print("✅ NLL calculator with caching loaded")
print("✅ Numerical integration utilities loaded")
print("\nKey Features:")
print("   • Dynamic integration range optimization")
print("   • NLL caching for 2-4x speedup")
print("   • Log-sum-exp for numerical stability")
print("   • Simpson's rule for accurate integration")
print("   • Automatic error handling and recovery")
print("=" * 70)

In [ ]:
# ==============================================================================
# SECTION 3: OPTIMIZED FTS Analysis - Hypothesis Testing with Toys
# ==============================================================================
# This section performs the main FTS analysis using the optimized implementation:
# 1. Creates hypothesis test points at different μ values
# 2. Computes observed test statistics from data using optimized FTS
# 3. Generates toy datasets to build null distributions
# 4. Calculates p-values for hypothesis testing
# ==============================================================================

import tqdm
import fts_core as FTS  # Import optimized FTS algorithms from src/fts_core.py

print("\n" + "=" * 70)
print("OPTIMIZED FTS Hypothesis Testing Analysis")
print("=" * 70)

analysis_start = time.time()

# ==============================================================================
# Create HypoSpace for Hypothesis Testing
# ==============================================================================
# HypoSpace is xRooFit's framework for managing hypothesis tests.
# It handles test points, toy generation, and p-value calculation.

print("\nSetting up hypothesis testing framework...")

# Create hypothesis space centered on the observed data
# pllType specifies the asymptotic approximation (Unknown = use toys)
# alt_value is the alternative hypothesis value
hs = w["pdfs/simPdf"].nll("obsData").hypoSpace(
    "mu",  # Parameter of interest
    pllType=ROOT.xRooFit.Asymptotics.Unknown,  # Use toys, not asymptotics
    alt_value=0  # Alternative hypothesis: μ=0 (background-only)
)

# ==============================================================================
# Configure Focus Function
# ==============================================================================
# Choose where to concentrate statistical power

focus = ProductionFocusFunction(
    mu_focus=0.0,        # Center focus at μ=0 (background-only hypothesis)
    sigma_focus=1.5,     # Width of focus region
    normalize=True,      # Normalize the focus function
)

# Why focus at μ=0?
# - For discovery: Focus near background-only improves sensitivity
# - For measurement: Focus near expected signal value
# - For limits: Candidate strategy — focus within the physically allowed region (evaluate per analysis)

# Create SINGLE NLL calculator instance for optimal caching
print("🔧 Creating optimized NLL calculator...")
nll_calc = CachedNLLCalculator(w)

# Create optimized FTS calculator
print("🔧 Creating optimized FTS calculator...")
opt_calc = FTS.OptimizedFTSCalculator()

# ==============================================================================
# Analysis Configuration
# ==============================================================================

# Define test points for hypothesis testing
test_mus = list(range(1, 10))  # Test μ = 1, 2, ..., 9

# Number of toy datasets per test point
# More toys = better p-value accuracy but longer runtime
num_toys = 100

# Integration grid density (optimized settings)
# Higher values = better numerical accuracy but slower
obs_integration_points = 201   # For observed data (high precision)
toy_integration_points = 101   # For toys (balanced speed/accuracy)

# Integration bounds aligned with parameter range
# These should match the physical bounds of μ
theta_bounds = (-100.0, 100.0)

# Display configuration
mu_range = focus.get_dynamic_integration_range(test_mus[0])
print(f"\n📊 Analysis Configuration:")
print(f"   • Test points: {len(test_mus)} values of μ")
print(f"   • Toys per point: {num_toys}")
print(f"   • Integration points: {obs_integration_points} (observed), {toy_integration_points} (toys)")
print(f"   • Focus: centered at μ={focus.mu_focus}, width σ={focus.sigma_focus}")
print(f"   • Parameter bounds: μ ∈ [{theta_bounds[0]}, {theta_bounds[1]}]")
print(f"   • Using optimized FTS implementation with proper caching")

# ==============================================================================
# Main Analysis Loop (OPTIMIZED VERSION)
# ==============================================================================
# For each test value of μ:
# 1. Compute observed test statistic using optimized FTS
# 2. Generate toys under null hypothesis
# 3. Build null distribution using optimized calculations
# 4. Calculate p-value

print("\n" + "=" * 70)
print("Starting OPTIMIZED hypothesis tests...")
print("=" * 70)

# Set random seed for reproducibility
base_seed = 123456

# Reset caches for clean performance measurement
FTS.reset_caches()

# Prepare containers for LRT observed/toy values
lrt_obs_values = []
lrt_toy_values = []  # list of lists per mu index

for mu_idx, mu in enumerate(test_mus):
    print(f"\n📍 Test Point {mu_idx+1}/{len(test_mus)}: μ = {mu}")
    print("-" * 40)
    
    point_start = time.time()
    # Initialize LRT toy container for this mu
    lrt_toy_values.append([])
    
    # ----------------------------------------------------------------------
    # Step 1: Create hypothesis point
    # ----------------------------------------------------------------------
    # Add a new test point to the hypothesis space
    hs.AddPoint(f"mu={mu}")
    hp = hs.back()  # Get reference to the new point
    
    # ----------------------------------------------------------------------
    # Step 2: Compute observed test statistic using OPTIMIZED FTS
    # ----------------------------------------------------------------------
    # This evaluates FTS for the actual data at μ=mu using the optimized algorithm
    
    print(f"Computing observed FTS (optimized)...")
    obs_ts = opt_calc.compute_fts(
        nll_calc,         # NLL calculator (persistent instance)
        "obsData",        # Dataset name
        mu0=float(mu),    # Null hypothesis value
        focus_obj=focus,  # Focus function
        theta_bounds=theta_bounds,  # Parameter bounds
        n_grid=obs_integration_points  # Integration precision
    )
    
    if obs_ts is None:
        print(f"  ❌ Failed to compute observed test statistic")
        continue
    
    # Store observed test statistic
    hp.setObsTS(obs_ts, 0.0)  # Second argument is auxiliary observable (unused)
    
    # Compute observed LRT (optimized)
    obs_lrt = opt_calc.compute_lrt(
        nll_calc,
        "obsData",
        float(mu),
        focus,
        theta_bounds,
        obs_integration_points,
        verbose=False
    )
    if obs_lrt is None:
        print("  ❌ Failed to compute observed LRT")
        continue
    lrt_obs_values.append(obs_lrt)
    print(f"  ✅ Observed FTS = {obs_ts:.3f}")
    
    # ----------------------------------------------------------------------
    # Step 3: Generate toy datasets and compute test statistics (OPTIMIZED)
    # ----------------------------------------------------------------------
    # Generate toys under the null hypothesis (μ=test value)
    # to build the null distribution for p-value calculation
    
    print(f"Generating {num_toys} toy datasets (optimized)...")
    
    # Fix μ to the test value for toy generation
    poi = w.pars()["mu"]
    old_val = poi.getVal()
    old_const = poi.isConstant()
    poi.setVal(float(mu))
    poi.setConstant(True)
    
    # Track successes and failures
    successful_toys = 0
    failed_toys = 0
    
    # Optimize minimizer settings for toys (faster, less precise)
    ROOT.Math.MinimizerOptions.SetDefaultStrategy(0)  # Fast strategy
    ROOT.Math.MinimizerOptions.SetDefaultTolerance(5e-3)  # Looser tolerance
    
    try:
        # Generate and process each toy using optimized FTS
        for i in tqdm.tqdm(range(num_toys), 
                          desc=f"  Processing toys for μ={mu}", 
                          leave=False,  # Don't leave progress bar after completion
                          ncols=80):    # Limit width for cleaner output
            try:
                # Set random seed for reproducibility
                FTS.set_root_seed(base_seed + i)
                
                # Generate toy dataset from model
                # This creates Poisson-distributed data based on the model
                toyset = w["pdfs/simPdf"].generate()
                toy_name = f"toy_mu{mu}_{i:04d}"
                toyset.get().SetNameTitle(toy_name, toy_name)
                w.Add(toyset)  # Add to workspace
                
                # Compute FTS for toy dataset using OPTIMIZED implementation
                toy_ts = opt_calc.compute_fts(
                    nll_calc,      # NLL calculator (SAME instance for caching)
                    toy_name,      # Toy dataset name
                    mu0=float(mu), # Null hypothesis (same as generation)
                    focus_obj=focus,  # Focus function
                    theta_bounds=theta_bounds,  # Parameter bounds
                    n_grid=toy_integration_points  # Lower precision for speed
                )
                
                # Compute LRT for toy dataset (optimized)
                toy_lrt = opt_calc.compute_lrt(
                    nll_calc,
                    toy_name,
                    float(mu),
                    focus,
                    theta_bounds,
                    toy_integration_points,
                    verbose=False
                )
                if toy_lrt is not None:
                    lrt_toy_values[mu_idx].append(toy_lrt)
                
                if toy_ts is not None:
                    # Add to null distribution
                    # Weight=1 means each toy contributes equally
                    hp.addNullToy(toy_ts, weight=1)
                    successful_toys += 1
                else:
                    failed_toys += 1
                
                # Clean up toy dataset to save memory
                try:
                    w.removeData(toy_name)
                    # Process ROOT events periodically to prevent UI freezing
                    if i % 50 == 0:
                        ROOT.gSystem.ProcessEvents()
                except:
                    pass  # Ignore cleanup errors
                    
            except Exception as e:
                # Count any failures but continue
                failed_toys += 1
        
    finally:
        # Always restore minimizer settings
        ROOT.Math.MinimizerOptions.SetDefaultStrategy(1)
        ROOT.Math.MinimizerOptions.SetDefaultTolerance(1e-3)
        poi.setVal(old_val)
        poi.setConstant(old_const)
    
    # ----------------------------------------------------------------------
    # Step 4: Report results for this test point (with optimization stats)
    # ----------------------------------------------------------------------
    
    point_time = time.time() - point_start
    perf_stats['point_time'].append(point_time)
    
    # Calculate success rate
    success_rate = successful_toys / max((successful_toys + failed_toys), 1) * 100.0
    
    # Get optimized cache statistics
    opt_stats = opt_calc.get_performance_stats()
    denom_cache_stats = opt_stats['denominator_cache']
    
    print(f"\n  📊 Results for μ={mu} (OPTIMIZED):")
    print(f"     • Time: {point_time:.1f}s")
    print(f"     • Toys: {successful_toys}/{num_toys} successful ({success_rate:.0f}%)")
    print(f"     • Denominator cache hit rate: {denom_cache_stats['hit_rate']:.1%}")
    print(f"     • Global grid cache entries: {opt_stats['global_grids_cached']}")
    print(f"     • Cache size: {denom_cache_stats['cache_size']} denominators")

# ==============================================================================
# Create Results Visualization
# ==============================================================================

print("\n" + "=" * 70)
print("Creating results visualization...")
print("=" * 70)

# Extract p-values from hypothesis space
# "pnulltoysreadonly" returns p-values computed from toy distributions
pnull_graph = hs.graph("pnulltoysreadonly")

# Configure plot appearance
pnull_graph.SetTitle("Optimized FTS p-values vs Signal Strength;#mu;p_{null}")
pnull_graph.SetMarkerStyle(20)  # Filled circles
pnull_graph.SetMarkerSize(1.2)
pnull_graph.SetLineWidth(2)
pnull_graph.SetMarkerColor(ROOT.kBlue)
pnull_graph.SetLineColor(ROOT.kBlue)

# Draw with logarithmic y-axis (common for p-value plots)
pnull_graph.Draw("ALP")  # A=Axis, L=Line, P=Points

# Build LRT p-values from collected toys
lrt_pvals = []
for k, mu in enumerate(test_mus):
    toys = lrt_toy_values[k] if k < len(lrt_toy_values) else []
    obs = lrt_obs_values[k] if k < len(lrt_obs_values) else None
    if obs is None or len(toys)==0:
        lrt_pvals.append(1.0)
    else:
        cnt = sum(1 for t in toys if t >= obs)
        lrt_pvals.append(cnt / len(toys))

# Create LRT TGraph and draw overlay
from array import array as _array
x_arr = _array("d", [float(x) for x in test_mus])
y_arr = _array("d", [float(max(min(p,1.0),1e-9)) for p in lrt_pvals])
lrt_graph = ROOT.TGraph(len(test_mus), x_arr, y_arr)
lrt_graph.SetMarkerStyle(24)
lrt_graph.SetMarkerSize(1.2)
lrt_graph.SetLineWidth(2)
lrt_graph.SetMarkerColor(ROOT.kRed)
lrt_graph.SetLineColor(ROOT.kRed)
lrt_graph.Draw("LP SAME")

# Add legend
leg = ROOT.TLegend(0.60, 0.75, 0.88, 0.88)
leg.SetBorderSize(0)
leg.SetFillStyle(0)
leg.AddEntry(pnull_graph, "FTS p_{null}", "lp")
leg.AddEntry(lrt_graph, "LRT p_{null}", "lp")
leg.Draw()
ROOT.gPad.SetLogy()      # Logarithmic y-axis
ROOT.gPad.GetCanvas().Draw()

# ==============================================================================
# Final Summary (OPTIMIZED VERSION)
# ==============================================================================

total_time = time.time() - analysis_start
perf_stats['total_analysis_time'].append(total_time)

# Get final optimization statistics
final_opt_stats = opt_calc.get_performance_stats()
final_denom_stats = final_opt_stats['denominator_cache']

print(f"\n🎉 OPTIMIZED Analysis Complete!")
print(f"   • Total time: {total_time/60:.1f} minutes")
print(f"   • Denominator cache performance: {final_denom_stats['hit_rate']:.1%} hit rate")
print(f"   • Total denominator calculations: {final_denom_stats['hits'] + final_denom_stats['misses']:,}")
print(f"   • Cache hits: {final_denom_stats['hits']:,}")
print(f"   • Cache misses: {final_denom_stats['misses']:,}")
print(f"   • Cached denominators: {final_denom_stats['cache_size']}")

# Compare with expected unoptimized performance
expected_old_time = 52.9  # minutes from previous runs
if total_time > 0:
    speedup = (expected_old_time * 60) / total_time
    print(f"\n🚀 Performance Improvement:")
    print(f"   • Estimated speedup: {speedup:.1f}x faster than original")
    print(f"   • Time saved: {expected_old_time - total_time/60:.1f} minutes")

print(f"\n📈 Focus configuration:")
print(f"   • Center: μ_focus = {focus.mu_focus}")
print(f"   • Width: σ_focus = {focus.sigma_focus}")
print(f"   • Integration bounds: [{theta_bounds[0]}, {theta_bounds[1]}]")

print(f"\n✅ Key Optimizations Applied:")
print(f"   • μ₀-independent global grid construction")
print(f"   • Pre-computed denominators with proper caching")
print(f"   • Persistent NLL calculator across all evaluations")
print(f"   • Optimized integration grid sizes")

print("=" * 70)

In [ ]:
# ==============================================================================
# SECTION 4: Results Analysis and Statistical Interpretation
# ==============================================================================
# This section extracts and analyzes the results:
# 1. Extract p-values from the hypothesis tests
# 2. Create summary table of results
# 3. Analyze the focus effect on sensitivity
# 4. Report performance metrics
# ==============================================================================

print("\n" + "=" * 70)
print("FTS Results Summary and Analysis")
print("=" * 70)

# ==============================================================================
# Extract P-values from HypoSpace
# ==============================================================================
# The p-value represents the probability of observing a test statistic
# at least as extreme as the observed value, assuming the null hypothesis

# Get p-value graph from hypothesis space
pnull_graph = hs.graph("pnulltoysreadonly")

# Extract (μ, p-value) pairs from the graph
import ctypes
x = ctypes.c_double(0.0)  # C-style double for ROOT interface
y = ctypes.c_double(0.0)
mu_to_p = {}

# Iterate through graph points
for i in range(int(pnull_graph.GetN())):
    pnull_graph.GetPoint(i, x, y)
    mu_to_p[float(x.value)] = float(y.value)

# ==============================================================================
# Create Results Summary Table
# ==============================================================================
print("\n📊 Hypothesis Test Results:")
print("=" * 50)
print(f"{'μ':^6} | {'Obs TS':^8} | {'p-value':^8} | {'Toys':^5}")
print("-" * 35)

# Helper functions for robust attribute extraction
import re, math

def _safe_float_attr(obj, names):
    """
    Safely extract float value from object attributes.
    Tries multiple possible attribute names for compatibility.
    """
    for name in names:
        attr = getattr(obj, name, None)
        if attr is None:
            continue
        try:
            # Handle both properties and methods
            return float(attr() if callable(attr) else attr)
        except Exception:
            continue
    return None

def _parse_mu_from_name(hp_obj):
    """
    Extract μ value from hypothesis point name.
    Names are formatted as "mu=X" where X is the value.
    """
    try:
        nm = hp_obj.GetName() if callable(getattr(hp_obj, 'GetName', None)) else ''
    except Exception:
        nm = ''
    # Use regex to extract numeric value after "mu="
    m = re.search(r"mu=([-+]?\d+(?:\.\d+)?)", str(nm))
    return float(m.group(1)) if m else float('nan')

# Collect and display results for each test point
valid_results = []

for i in range(hs.size()):
    hp = hs[i]  # Get hypothesis point
    
    # Extract μ value (try multiple methods for robustness)
    mu_val = _safe_float_attr(hp, ['fNullVal', 'fNull', 'nullVal'])
    if mu_val is None:
        mu_val = _parse_mu_from_name(hp)

    # Extract observed test statistic
    obs_ts = _safe_float_attr(hp, ['obsTS', 'obs_ts', 'tObs', 'tsObs'])
    if obs_ts is None:
        obs_ts = float('nan')

    # Get p-value from extracted graph data
    pval = mu_to_p.get(mu_val)
    
    # Handle floating point precision issues
    if pval is None and math.isfinite(mu_val) and len(mu_to_p) > 0:
        # Find closest μ value in case of rounding differences
        xs = sorted(mu_to_p.keys(), key=lambda x: abs(x - mu_val))
        if abs(xs[0] - mu_val) < 1e-8:
            pval = mu_to_p[xs[0]]
    
    if pval is None:
        pval = -1.0  # Mark as unavailable

    # Count number of successful toys
    try:
        n_toys = len(hp.nullToys().get()) if hp.nullToys().get() else 0
    except Exception:
        n_toys = 0

    # Store results
    valid_results.append((mu_val, obs_ts, pval, n_toys))

    # Display in table
    if pval >= 0:
        print(f"{mu_val:^6.0f} | {obs_ts:^8.3f} | {pval:^8.4f} | {n_toys:^5.0f}")
    else:
        print(f"{mu_val:^6.0f} | {obs_ts:^8.3f} | {'N/A':^8} | {n_toys:^5.0f}")

# ==============================================================================
# Analyze Focus Effect
# ==============================================================================
# Compare sensitivity in the focus region vs outside to quantify improvement

print("\n" + "=" * 50)
print("Focus Effect Analysis")
print("=" * 50)

# Filter results with valid p-values
valid_pval_results = [(mu, ts, pv, n) for mu, ts, pv, n in valid_results if pv >= 0]

# Categorize points by distance from focus center
# Inside focus: within 1σ of focus center
focus_points = [(mu, ts, pv, n) for mu, ts, pv, n in valid_pval_results 
                if abs(mu - focus.mu_focus) <= focus.sigma_focus]

# Outside focus: beyond 2σ from focus center
outside_points = [(mu, ts, pv, n) for mu, ts, pv, n in valid_pval_results 
                  if abs(mu - focus.mu_focus) > 2*focus.sigma_focus]

if focus_points and outside_points:
    # Calculate average p-values in each region
    focus_pval_avg = sum(pv for _, _, pv, _ in focus_points) / len(focus_points)
    outside_pval_avg = sum(pv for _, _, pv, _ in outside_points) / len(outside_points)
    
    print(f"\n📍 Focus region (|μ - {focus.mu_focus}| ≤ {focus.sigma_focus}):")
    print(f"   • Points: {len(focus_points)}")
    print(f"   • Average p-value: {focus_pval_avg:.4f}")
    
    print(f"\n📍 Outside region (|μ - {focus.mu_focus}| > {2*focus.sigma_focus}):")
    print(f"   • Points: {len(outside_points)}")  
    print(f"   • Average p-value: {outside_pval_avg:.4f}")
    
    # Quantify sensitivity improvement
    if focus_pval_avg < outside_pval_avg:
        improvement = outside_pval_avg / focus_pval_avg
        print(f"\n✨ Focus Effect Detected:")
        print(f"   • {improvement:.1f}x better sensitivity in focus region")
        print(f"   • Lower p-values indicate stronger evidence against null")
    else:
        print(f"\n⚠️  No significant focus effect observed")
        print(f"   • May need to adjust focus parameters")

# ==============================================================================
# Performance Statistics
# ==============================================================================
print("\n" + "=" * 50)
print("Performance Metrics")
print("=" * 50)

final_cache_stats = nll_calc.get_cache_stats()

print(f"\n⏱️  Timing:")
print(f"   • Total analysis: {perf_stats['total_analysis_time'][0]/60:.1f} minutes")
print(f"   • Setup time: {perf_stats['setup_time'][0]:.2f} seconds")
if perf_stats['point_time']:
    print(f"   • Average per test point: {np.mean(perf_stats['point_time']):.1f} seconds")

print(f"\n💾 Cache Performance:")
print(f"   • Hit rate: {final_cache_stats['hit_rate']:.0%}")
print(f"   • Total NLL evaluations: {final_cache_stats['hits'] + final_cache_stats['misses']:,}")
print(f"   • Cache hits: {final_cache_stats['hits']:,}")
print(f"   • Cache misses: {final_cache_stats['misses']:,}")
print(f"   • Failed fits: {final_cache_stats['failed_fits']}")

print(f"\n🔧 Efficiency Gains:")
if final_cache_stats['hit_rate'] > 0:
    speedup = 1 / (1 - final_cache_stats['hit_rate'])
    print(f"   • Effective speedup from caching: {speedup:.1f}x")
    saved_time = perf_stats['total_analysis_time'][0] * (1 - 1/speedup) / 60
    print(f"   • Time saved: ~{saved_time:.1f} minutes")

# ==============================================================================
# Implementation Summary
# ==============================================================================
print("\n" + "=" * 50)
print("✅ FTS Implementation Complete")
print("=" * 50)

print("\nKey achievements:")
print("   • Successfully implemented FTS algorithm")
print("   • Generated null distributions from toys")
print("   • Computed p-values for hypothesis testing")
print("   • Demonstrated focus effect on sensitivity")
print("   • Achieved efficient computation with caching")

print("\nNext steps:")
print("   • Compare with traditional likelihood ratio test")
print("   • Optimize focus parameters for specific physics goals")
print("   • Apply to real experimental data")
print("   • Compute confidence intervals and limits")

print("\n" + "=" * 70)

In [ ]:
# ==============================================================================
# SECTION 5: Detailed Visualization of Best-Fit Point
# ==============================================================================
# This section creates a detailed visualization of the hypothesis point
# with the lowest p-value (most significant result), showing:
# - The null distribution from toys
# - The observed test statistic
# - The p-value calculation
# ==============================================================================

import ctypes, ROOT, math

print("\n" + "=" * 70)
print("Visualizing Most Significant Result")
print("=" * 70)

# ==============================================================================
# Find the Most Significant Test Point
# ==============================================================================
# The most significant point has the smallest p-value,
# indicating the strongest evidence against the null hypothesis

# Get p-value graph
pnull_graph = hs.graph("pnulltoysreadonly")

# Find minimum p-value
x = ctypes.c_double(0.0)
y = ctypes.c_double(0.0)
best_i, best_p = 0, 1.0
best_mu = 0.0

for i in range(int(pnull_graph.GetN())):
    pnull_graph.GetPoint(i, x, y)
    if y.value < best_p:
        best_p = float(y.value)
        best_i = i
        best_mu = float(x.value)

print(f"\n📊 Most significant result:")
print(f"   • Test point: μ = {best_mu}")
print(f"   • p-value: {best_p:.4f}")
print(f"   • Hypothesis point index: {best_i}")

# ==============================================================================
# Create Detailed Visualization
# ==============================================================================
# This plot shows:
# 1. Histogram of test statistics from toys (null distribution)
# 2. Vertical line showing observed test statistic
# 3. Shaded region showing p-value

# Get the hypothesis point
hp = hs[best_i]

# Create canvas for plotting
c = ROOT.TCanvas("c_hp", f"FTS Distribution for μ = {best_mu}", 800, 600)

# Draw the hypothesis point
# This creates a histogram of toy test statistics with observed value marked
hp.Draw()

# Configure plot appearance
ROOT.gPad.SetLogy()  # Log scale for y-axis (useful for tail visualization)
ROOT.gPad.SetGridx()  # Add vertical grid lines
ROOT.gPad.SetGridy()  # Add horizontal grid lines

# Update canvas
ROOT.gPad.GetCanvas().Draw()

# ==============================================================================
# Interpretation Guide
# ==============================================================================
print("\n" + "=" * 70)
print("📈 Plot Interpretation:")
print("=" * 70)

print("""
The plot shows the null distribution and observed test statistic:

1. **Histogram (Blue)**: Distribution of FTS values from toy datasets
   - Generated under null hypothesis (μ = {:.1f})
   - Shows expected variation due to statistical fluctuations
   
2. **Vertical Line (Red)**: Observed FTS value from actual data
   - Position indicates how extreme the observation is
   - Further right = more inconsistent with null hypothesis
   
3. **P-value**: Fraction of toys with FTS ≥ observed value
   - p = {:.4f} for this test point
   - Small p-value = strong evidence against null hypothesis
   
4. **Focus Effect**: 
   - FTS concentrates power near μ = {:.1f}
   - Compare with standard LRT to see improvement
   
Statistical Interpretation:
""".format(best_mu, best_p, focus.mu_focus))

# Provide interpretation based on p-value
if best_p < 0.003:
    print("   ⭐ 3σ evidence or stronger (p < 0.003)")
    print("   → Strong evidence against the null hypothesis")
elif best_p < 0.05:
    print("   ⭐ 2σ evidence (p < 0.05)")
    print("   → Moderate evidence against the null hypothesis")
elif best_p < 0.32:
    print("   ⭐ 1σ evidence (p < 0.32)")
    print("   → Weak evidence against the null hypothesis")
else:
    print("   ⭐ No significant evidence (p ≥ 0.32)")
    print("   → Data consistent with null hypothesis")

print("\n" + "=" * 70)
print("✅ Visualization complete!")
print("=" * 70)

In [ ]:
# ==============================================================================
# SECTION 6: OPTIMIZED FTS IMPLEMENTATION DEMONSTRATION
# ==============================================================================
# This section demonstrates the optimized FTS implementation that addresses
# the performance issues identified in detailed performance analysis:
# 1. Eliminates redundant denominator calculations
# 2. Implements proper caching with distinct dataset keys
# 3. Uses μ₀-independent global grids
# 4. Provides fair FTS vs LRT comparison
# ==============================================================================

print("\n" + "=" * 70)
print("FTS OPTIMIZATION DEMONSTRATION")
print("=" * 70)

# Import optimized functions from updated fts_core
from fts_core import (
    fts_ts_obs_optimized, lrt_ts_optimized, 
    get_optimization_stats, reset_caches,
    OptimizedFTSCalculator
)

# ==============================================================================
# Create Optimized Calculator Instance
# ==============================================================================
# This replaces the ad-hoc calculations with a systematic approach

print("\n🔧 Creating optimized FTS calculator...")
opt_calc = OptimizedFTSCalculator()

# Reset any existing caches for clean comparison
reset_caches()

# ==============================================================================
# Performance Comparison: Old vs Optimized
# ==============================================================================
print("\n📊 Performance Comparison:")
print("-" * 50)

# Test on a subset of μ values for speed
test_points = [1.0, 2.0, 3.0, 4.0]
comparison_results = {
    'mu_values': [],
    'fts_optimized': [],
    'lrt_optimized': [],
    'time_per_point': []
}

print("Testing optimized implementation on sample points...")

# Create a persistent NLL calculator for fair comparison
# This is crucial: same instance across all μ₀ evaluations
persistent_nll_calc = CachedNLLCalculator(w)

for mu0 in test_points:
    point_start = time.time()
    
    print(f"  • Testing μ₀ = {mu0}...")
    
    # Compute FTS and LRT using optimized methods
    fts_val = opt_calc.compute_fts(
        persistent_nll_calc, "obsData", mu0, focus, 
        theta_bounds=(-100.0, 100.0), n_grid=201  # Smaller grid for demo
    )
    
    lrt_val = opt_calc.compute_lrt(
        persistent_nll_calc, "obsData", mu0, focus,
        theta_bounds=(-100.0, 100.0), n_grid=201
    )
    
    point_time = time.time() - point_start
    
    comparison_results['mu_values'].append(mu0)
    comparison_results['fts_optimized'].append(fts_val)
    comparison_results['lrt_optimized'].append(lrt_val)
    comparison_results['time_per_point'].append(point_time)
    
    print(f"    FTS = {fts_val:.4f}, LRT = {lrt_val:.4f}, Time = {point_time:.2f}s")

# ==============================================================================
# Validation Tests (Addressing Technical Analysis)
# ==============================================================================
print("\n" + "=" * 50)
print("VALIDATION TESTS")
print("=" * 50)

# Test 1: Denominator invariance
print("\n✅ Test 1: Denominator Invariance")
print("Testing that denominator doesn't change with μ₀...")

test_mu0_values = [0.5, 1.0, 1.5, 2.0]
denominators = []

for mu0 in test_mu0_values:
    # Access internal denominator calculation
    grid_key = (focus.mu_focus, focus.sigma_focus, (-100.0, 100.0), 201)
    
    # Get cached denominator entry
    if hasattr(opt_calc.global_grid_cache, '__contains__') and grid_key in opt_calc.global_grid_cache:
        mu_grid, w_grid, grid_info = opt_calc.global_grid_cache[grid_key]
        denom_entry = opt_calc.denominator_cache.get_denominator(
            "obsData", focus, persistent_nll_calc, grid_info, mu_grid, w_grid
        )
        denominators.append(denom_entry['log_denom'])
    else:
        # Force computation by calling compute_fts
        opt_calc.compute_fts(persistent_nll_calc, "obsData", mu0, focus, (-100.0, 100.0), 201)
        # Get the cached result
        mu_grid, w_grid, grid_info = opt_calc.global_grid_cache[grid_key]
        denom_entry = opt_calc.denominator_cache.get_denominator(
            "obsData", focus, persistent_nll_calc, grid_info, mu_grid, w_grid
        )
        denominators.append(denom_entry['log_denom'])

if len(denominators) > 1:
    denom_std = np.std(denominators)
    denom_mean = np.mean(denominators)
    relative_std = denom_std / abs(denom_mean) if denom_mean != 0 else float('inf')
    
    print(f"Denominators: {denominators}")
    print(f"Standard deviation: {denom_std:.2e}")
    print(f"Relative std: {relative_std:.2e}")
    
    if relative_std < 1e-9:
        print("✓ PASS: Denominator is invariant to μ₀")
    else:
        print("✗ FAIL: Denominator varies with μ₀")
else:
    print("⚠ Could not test denominator invariance")

# Test 2: Constant offset property
print("\n✅ Test 2: FTS-LRT Constant Offset")
print("Testing theoretical relationship: FTS ≈ LRT + constant...")

if len(comparison_results['fts_optimized']) >= 2:
    fts_vals = np.array(comparison_results['fts_optimized'])
    lrt_vals = np.array(comparison_results['lrt_optimized'])
    
    offsets = fts_vals - lrt_vals
    offset_std = np.std(offsets)
    offset_mean = np.mean(offsets)
    
    print(f"FTS values: {fts_vals}")
    print(f"LRT values: {lrt_vals}")
    print(f"Offsets (FTS-LRT): {offsets}")
    print(f"Mean offset: {offset_mean:.4f}")
    print(f"Offset std: {offset_std:.4f}")
    
    if offset_std < 0.01:
        print("✓ PASS: Constant offset property holds")
        print(f"  Theoretical offset ≈ {offset_mean:.3f}")
    else:
        print("✗ FAIL: Offset is not constant")
else:
    print("⚠ Need more data points to test constant offset")

# Test 3: Cache performance
print("\n✅ Test 3: Cache Performance")
cache_stats = opt_calc.get_performance_stats()
print(f"Cache statistics: {cache_stats}")

denom_cache_stats = cache_stats['denominator_cache']
if denom_cache_stats['hit_rate'] > 0.5:
    print(f"✓ PASS: Good cache hit rate ({denom_cache_stats['hit_rate']:.1%})")
else:
    print(f"⚠ Suboptimal cache hit rate: {denom_cache_stats['hit_rate']:.1%}")
    
print(f"Grid cache entries: {cache_stats['global_grids_cached']}")

# ==============================================================================
# Create Comparison Visualization
# ==============================================================================
print("\n📊 Creating FTS vs LRT comparison plot...")

import matplotlib.pyplot as plt

# Set up the plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Plot 1: FTS vs LRT values
mu_vals = comparison_results['mu_values']
fts_vals = comparison_results['fts_optimized']
lrt_vals = comparison_results['lrt_optimized']

ax1.scatter(mu_vals, fts_vals, color='red', label='FTS', s=100, alpha=0.7)
ax1.scatter(mu_vals, lrt_vals, color='blue', label='LRT', s=100, alpha=0.7)
ax1.plot(mu_vals, fts_vals, 'r--', alpha=0.5)
ax1.plot(mu_vals, lrt_vals, 'b--', alpha=0.5)

ax1.set_xlabel('μ₀ (null hypothesis)', fontsize=12)
ax1.set_ylabel('Test Statistic Value', fontsize=12)
ax1.set_title('FTS vs LRT Comparison', fontsize=14, fontweight='bold')
ax1.legend(fontsize=11)
ax1.grid(True, alpha=0.3)

# Plot 2: Constant offset demonstration
if len(fts_vals) >= 2:
    offsets = np.array(fts_vals) - np.array(lrt_vals)
    ax2.scatter(mu_vals, offsets, color='green', s=100)
    ax2.axhline(y=np.mean(offsets), color='green', linestyle='-', 
                label=f'Mean offset = {np.mean(offsets):.3f}')
    ax2.axhline(y=np.mean(offsets) + np.std(offsets), color='green', 
                linestyle='--', alpha=0.5, label=f'±1σ = ±{np.std(offsets):.3f}')
    ax2.axhline(y=np.mean(offsets) - np.std(offsets), color='green', 
                linestyle='--', alpha=0.5)
    
    ax2.set_xlabel('μ₀ (null hypothesis)', fontsize=12)
    ax2.set_ylabel('FTS - LRT', fontsize=12)
    ax2.set_title('Constant Offset Property', fontsize=14, fontweight='bold')
    ax2.legend(fontsize=10)
    ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# ==============================================================================
# Performance Summary
# ==============================================================================
print("\n" + "=" * 70)
print("OPTIMIZATION SUMMARY")
print("=" * 70)

avg_time_per_point = np.mean(comparison_results['time_per_point'])
total_test_time = sum(comparison_results['time_per_point'])

print(f"✅ Optimized Implementation Results:")
print(f"   • Test points evaluated: {len(comparison_results['mu_values'])}")
print(f"   • Average time per point: {avg_time_per_point:.2f}s")
print(f"   • Total computation time: {total_test_time:.2f}s")
print(f"   • Cache hit rate: {denom_cache_stats['hit_rate']:.1%}")

# Estimate performance improvement
old_avg_time = 352.7  # From previous cell output
if avg_time_per_point > 0:
    speedup_factor = old_avg_time / avg_time_per_point
    print(f"   • Estimated speedup: {speedup_factor:.1f}x faster")
    print(f"   • Projected full analysis time: {(old_avg_time * 9) / speedup_factor / 60:.1f} minutes")
